www.bigrabbitdata.com

In [1]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm

## Mapping each unique word to an unique index

In [2]:
class Dictionary(object):
    '''
        Create a dictionary, map each unique word to an unique index
    '''
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
            
    def __len__(self):
        return len(self.word2idx)

## Process Text to get trainable input 

In [3]:
class TextProcess(object):
    '''
        process the txt file and return batched-trainable data
    '''
    
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size=20):
        # add all the unique words to the dictionary
        with open(path, 'r') as f:
            total_tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                total_tokens += len(words)
                for word in words: 
                    self.dictionary.add_word(word) 
        
        # Create a empty 1-D tensor that contains the index of all the words in the text file
        input_tensor = torch.LongTensor(total_tokens)
        index = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    input_tensor[index] = self.dictionary.word2idx[word]
                    index += 1
                    
        # Find out how many batches we have          
        num_batches = total_tokens // batch_size
        
        #Remove the leftover that doesn't fill a batch
        input_tensor = input_tensor[:num_batches*batch_size]
        # return (batch_size, size of each batch)
        input_tensor = input_tensor.view(batch_size, -1)
        return input_tensor, total_tokens

In [4]:
embed_size = 128    # Input features to the LSTM
hidden_size = 1024  # Number of LSTM units
num_layers = 1      # Number of layer stacked  
num_epochs = 50
batch_size = 20
timesteps = 30
learning_rate = 0.002

corpus = TextProcess()

input_tensor, total_tokens = corpus.get_data('earnest.txt', batch_size)
print ("we have {} total words in our text".format(total_tokens))

print("(batch_size, number of words for each batch) = ", input_tensor.shape)
vocab_size = len(corpus.dictionary)
print ("We have {} unique words in our dictionary".format(vocab_size))

num_batches = input_tensor.shape[1] // timesteps
print("number of batches {}".format(num_batches))

we have 27226 total words in our text
(batch_size, number of words for each batch) =  torch.Size([20, 1361])
We have 4930 unique words in our dictionary
number of batches 45


## LSTM Model 

In [5]:
class TextGenerator(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(TextGenerator, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, prev_state):
        embed = self.embed(x)
        output, (hn, cn) = self.lstm(embed, prev_state)
        # output shape is (batch_size, timestep, hidden_size) = (20, 30, 1024)
        
        # reshape the output to match our fully connected layer
        output = output.reshape(-1, output.size(2))
        # (-1, 1024)
        
        output = self.linear(output)
        return output, (hn, cn)

## Training in action

In [6]:
# Truncated backpropagation
def detach(states):
    return [state.detach() for state in states] 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = TextGenerator(vocab_size, embed_size, hidden_size, num_layers).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))

    for i in range(0, input_tensor.size(1) - timesteps, timesteps):
        # Get mini-batch inputs and targets
        inputs = input_tensor[:, i:i+timesteps].to(device)
        targets = input_tensor[:, (i+1):(i+1)+timesteps].to(device)
        
        states = detach(states)
        outputs,_ = model(inputs, states)
        loss = loss_fn(outputs, targets.reshape(-1))

        model.zero_grad()
        loss.backward()
        # Perform Gradient Clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
    
    else:
        print ('Epoch [{}/{}], Loss: {:.4f}'
           .format(epoch+1, num_epochs, loss.item()))

Epoch [1/50], Loss: 6.4108
Epoch [2/50], Loss: 5.4994
Epoch [3/50], Loss: 4.7959
Epoch [4/50], Loss: 4.1645
Epoch [5/50], Loss: 3.6922
Epoch [6/50], Loss: 3.1443
Epoch [7/50], Loss: 2.7539
Epoch [8/50], Loss: 2.4563
Epoch [9/50], Loss: 2.0024
Epoch [10/50], Loss: 1.6632
Epoch [11/50], Loss: 1.3270
Epoch [12/50], Loss: 0.9071
Epoch [13/50], Loss: 0.6703
Epoch [14/50], Loss: 0.4337
Epoch [15/50], Loss: 0.2859
Epoch [16/50], Loss: 0.1890
Epoch [17/50], Loss: 0.1323
Epoch [18/50], Loss: 0.0903
Epoch [19/50], Loss: 0.0820
Epoch [20/50], Loss: 0.0787
Epoch [21/50], Loss: 0.0759
Epoch [22/50], Loss: 0.0764
Epoch [23/50], Loss: 0.0737
Epoch [24/50], Loss: 0.0750
Epoch [25/50], Loss: 0.0723
Epoch [26/50], Loss: 0.0742
Epoch [27/50], Loss: 0.0712
Epoch [28/50], Loss: 0.0736
Epoch [29/50], Loss: 0.0703
Epoch [30/50], Loss: 0.0732
Epoch [31/50], Loss: 0.0697
Epoch [32/50], Loss: 0.0727
Epoch [33/50], Loss: 0.0692
Epoch [34/50], Loss: 0.0722
Epoch [35/50], Loss: 0.0688
Epoch [36/50], Loss: 0.0718
E

## Generate random sentences

In [18]:
# Test the model
with torch.no_grad():
    random_sentences = ''

    # Set intial hidden ane cell states
    state = (torch.zeros(num_layers, 1, hidden_size).to(device),
             torch.zeros(num_layers, 1, hidden_size).to(device))
    # Select one word index randomly and convert it to shape (1,1)
    input = torch.randint(0,vocab_size, (1,)).long().unsqueeze(1).to(device)
    print ("randomly selected word index ", input.item())

    for i in range(20):
        output, _ = model(input, state)
       
        # we want our output to be 
        prob = output.exp()
    
        word_id = torch.multinomial(prob, num_samples=1).item()

        
        # Replace the input with sampled word id for the next time step
        input.fill_(word_id)
    
        word = corpus.dictionary.idx2word[word_id]
        word = '\n' if word == '<eos>' else word + ' '
        random_sentences += word
    print (random_sentences)


randomly selected word index  1635

[Enter Lady Bracknell. [Sternly.] Mr. Worthing, 
Jack. Darling! International letters to me if any kind. 
Gwendolen. 

